# Creating and Loading tables from csv files

In [ ]:
import csv
import pprint
data_list = [
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_StatisticalAreas.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_BusinessStats.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_CarSharingPods.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_CarSharingPods_py.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_CensusStats.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/new_Neighbourhoods.csv'))),
    list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/result.csv')))
]
pprint.pprint(data_list[0][3])

In [ ]:
def pgexec( conn, sqlcmd, args, msg, silent=False ):
   """ utility function to execute some SQL statement
       can take optional arguments to fill in (dictionary)
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
               cur.execute(sqlcmd)
            else:
               cur.execute(sqlcmd, args)
            if silent == False: 
                print("success: " + msg)
            retval = True
         except Exception as e:
            if silent == False: 
                print("db error: ")
                print(e)
   return retval

In [ ]:
def pgquery( conn, sqlcmd, args, silent=False ):
   """ utility function to execute some SQL query statement
       can take optional arguments to fill in (dictionary)
       will print out on screen the result set of the query
       error and transaction handling built-in """
   retval = False
   with conn:
      with conn.cursor() as cur:
         try:
            if args is None:
                cur.execute(sqlcmd)
            else:
                cur.execute(sqlcmd, args)
            if silent == False:
                for record in cur:
                    print(record)
            retval = True
         except Exception as e:
            if silent == False:
                print("db read error: ")
                print(e)
   return retval

In [ ]:
import psycopg2

def pgconnect():
    # please replace <your_unikey> and <your_SID> with your own details
    YOUR_UNIKEY = 'hohe6356'#'<your_unikey>'
    YOUR_PW     = '460012791'#'<your_SID>'
    try: 
        conn = psycopg2.connect(host='soit-db-pro-1.ucc.usyd.edu.au',
                               database='y18s1d2001_'+YOUR_UNIKEY,
                               user='y18s1d2001_'+YOUR_UNIKEY, 
                               password=YOUR_PW)
        
#         conn = psycopg2.connect(host='localhost', database='waterinfo', user='matloob', password='matloob')
        
        print('connected')
    except Exception as e:
        print("unable to connect to the database")
        print(e)
    return conn

In [ ]:
# 1st: login to database
conn = pgconnect()

# if you want to reset the table
pgexec (conn, 'DROP VIEW IF EXISTS carpods',None, "Drop view")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.BusinessStats',None, "Reset Table Organisation")
pgexec (conn, 'DROP TABLE IF EXISTS Walkability.CarSharingPods',None, "Reset Table CarSharingPods")
pgexec (conn, 'DROP TABLE IF EXISTS Walkability.CarSharingPods_py',None, "Reset Table CarSharingPods_py")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.CensusStats',None, "Reset Table CensusStats")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.Neighbourhoods',None, "Reset Table Neighbourhoods")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.Result',None, "Reset Table Result")
pgexec (conn, 'DROP TABLE IF EXISTS walkability.StatisticalAreas',None, "Reset Table StatisticalAreas")


# 2nd: ensure that the schema is in place
StatisticalAreas_schema = """CREATE TABLE IF NOT EXISTS walkability.StatisticalAreas(
                        area_id BIGINT PRIMARY KEY,
                        area_name TEXT,
                        parent_area_id INT                        
                        )"""
CarSharingPods_schema = """CREATE TABLE IF NOT EXISTS Walkability.CarSharingPods (
                        pod_id BIGINT PRIMARY KEY,
                        name TEXT,
                        num_cars INT,
                        gps POINT,
                        description TEXT
                        )"""
CarSharingPods_py_schema = """CREATE TABLE IF NOT EXISTS Walkability.CarSharingPods_py (
                        pod_id BIGINT PRIMARY KEY,
                        name TEXT,
                        num_cars INT,
                        gps POINT,
                        description TEXT,
                        area_id BIGINT
                        )"""
BusinessStats_schema = """CREATE TABLE IF NOT EXISTS walkability.BusinessStats (
                        area_id BIGINT REFERENCES walkability.StatisticalAreas(area_id) PRIMARY KEY,
                        num_businesses FLOAT,
                        retail_trade FLOAT,
                        accommodation_and_food_services FLOAT,
                        health_care_and_social_assistance FLOAT,
                        education_and_training FLOAT,
                        arts_and_recreation_services FLOAT
                        )"""

CensusStats_schema = """CREATE TABLE IF NOT EXISTS walkability.CensusStats(
                        area_id BIGINT REFERENCES walkability.StatisticalAreas(area_id) PRIMARY KEY,
                        median_annual_household_income FLOAT,
                        avg_monthly_rent FLOAT
                        )"""
Neighbourhoods_schema = """CREATE TABLE IF NOT EXISTS walkability.Neighbourhoods (
                        area_id BIGINT REFERENCES walkability.StatisticalAreas(area_id) PRIMARY KEY,
                        area_name TEXT,
                        land_area FLOAT,
                        population FLOAT,
                        number_of_dwellings INT,
                        number_of_businesses FLOAT,
                        gps POINT,
                        boundary BOX
                        )"""
result_schema = """CREATE TABLE IF NOT EXISTS walkability.result (
                        area_id BIGINT REFERENCES walkability.StatisticalAreas(area_id) PRIMARY KEY,
                        area_name TEXT,
                        population_density FLOAT,
                        population_density_Z FLOAT,
                        dwelling_density FLOAT,
                        dwelling_density_Z FLOAT,
                        transport_density FLOAT,
                        transport_density_Z FLOAT,
                        service_balance FLOAT,
                        retail_trade_z FLOAT,
                        accommodation_and_food_services_z FLOAT,
                        health_care_and_social_assistance_z FLOAT,
                        education_and_training_z FLOAT,
                        arts_and_recreation_services_z FLOAT,
                        walkability INT
                        )"""

pgexec (conn, StatisticalAreas_schema, None, 'Create Table StatisticalAreas')
pgexec (conn, BusinessStats_schema, None, 'Create Table BusinessStats')
pgexec (conn, CarSharingPods_schema, None, 'Create Table CarSharingPods')
pgexec (conn, CarSharingPods_py_schema, None, 'Create Table CarSharingPods')
pgexec (conn, CensusStats_schema, None, 'Create Table CensusStats')
pgexec (conn, Neighbourhoods_schema, None, 'Create Table Neighbourhoods')
pgexec (conn, result_schema, None, 'Create Table Result')


# 3nd: load data
# IMPORTANT: make sure the header line of CSV is without spaces!
insert_stmt =[
    """INSERT INTO walkability.StatisticalAreas(area_id,area_name,parent_area_id) VALUES (%(area_id)s,%(area_name)s,%(parent_area_id)s)""",
    """INSERT INTO walkability.BusinessStats(area_id,num_businesses,retail_trade,accommodation_and_food_services,health_care_and_social_assistance,education_and_training,arts_and_recreation_services) VALUES (%(area_id)s,%(num_businesses)s,%(retail_trade)s,%(accommodation_and_food_services)s,%(health_care_and_social_assistance)s,%(education_and_training)s,%(arts_and_recreation_services)s)""",
    """INSERT INTO Walkability.CarSharingPods(pod_id,name,num_cars,gps,description) VALUES (%(\ufeffpod_id)s,%(name)s,%(num_cars)s,PoinT(%(latitude)s,%(longitude)s),%(description)s)""",
    """INSERT INTO Walkability.CarSharingPods_py(pod_id,name,num_cars,gps,description,area_id) VALUES (%(pod_id)s,%(name)s,%(num_cars)s,PoinT(%(latitude)s,%(longitude)s),%(description)s,%(area_id)s)""",
    """INSERT INTO walkability.CensusStats(area_id,median_annual_household_income,avg_monthly_rent) VALUES (%(area_id)s,%(median_annual_household_income)s,%(avg_monthly_rent)s)""",
    """INSERT INTO walkability.Neighbourhoods(area_id,area_name,land_area,population,number_of_dwellings,number_of_businesses,gps,boundary) VALUES (%(area_id)s,%(area_name)s,%(land_area)s,%(population)s,%(number_of_dwellings)s,%(number_of_businesses)s,POINT(%(GPS_lat)s,%(GPS_lng)s),BOX(Point(%(Location_northeast_lat)s,%(Location_northeast_lng)s),Point(%(Location_southwest_lat)s,%(Location_southwest_lng)s)))""",
    """INSERT INTO walkability.result(area_id,area_name,population_density,dwelling_density,transport_density,retail_trade_z,accommodation_and_food_services_z,health_care_and_social_assistance_z,education_and_training_z,arts_and_recreation_services_z,population_density_Z,dwelling_density_Z,transport_density_Z,walkability,service_balance) 
    VALUES (%(area_id)s,%(area_name)s,%(population_density)s,%(dwelling_density)s,%(transport_density)s,%(retail_trade_z)s,%(accommodation_and_food_services_z)s,%(health_care_and_social_assistance_z)s,%(education_and_training_z)s,%(arts_and_recreation_services_z)s,%(population_density_z)s,%(dwelling_density_z)s,%(transport_density_z)s,%(walkability)s,%(service_balance)s)"""
]
i=1
for insert,data in zip(insert_stmt,data_list):
    for row in data:
         pgexec (conn, insert, row, "row inserted")
         print(i)
    i += 1

# Adding columns to tables 

In [ ]:
# Add a new column called area_id in the table carsharingpods
# conn = pgconnect()
sqlcmd = """ALTER TABLE walkability.carsharingpods 
             ADD COLUMN area_id BIGINT REFERENCES walkability.StatisticalAreas(area_id);"""
pgquery(conn, sqlcmd, None)

# Fill in the area_id column with the result returned from spatial join
sqlcmd = """UPDATE walkability.carsharingpods C
               SET (area_id) = (SELECT MAX(area_id) FROM walkability.neighbourhoods N
                                 WHERE N.boundary @> C.gps);"""
pgquery(conn, sqlcmd, None)

In [ ]:
# Add a new column called walkability in the table neighbourhoods
conn = pgconnect()
sqlcmd = """ALTER TABLE walkability.neighbourhoods 
             ADD COLUMN walkability FLOAT;"""
pgquery(conn, sqlcmd, None)

# Fill in the walkability column with the data from csv file which stores the walkability 
data = list(csv.DictReader(open('/home/hohe6356/Desktop/files_csv/Walkability_final_result.csv')))

# Step 1. Create a temporary table to load csv file
pgexec(conn,"""DROP TABLE IF EXISTS tmp;""",None,"tmp table reset")
tmp_schema = """CREATE TABLE IF NOT EXISTS tmp(area_id BIGINT, walkability FLOAT);"""
pgexec(conn, tmp_schema, None, "tmp table created")

# Step 2. Load the temporary table with the value from csv
insert_stmt = """INSERT INTO tmp(area_id,walkability) VALUES (%(area_id)s,%(walkability)s)"""
for row in data:
     pgexec (conn, insert_stmt, row, "row inserted")

# Step 3. Update the neighbourhoods table using tmp
sqlcmd = """UPDATE walkability.neighbourhoods 
               SET walkability = T.walkability
              FROM tmp T
             WHERE neighbourhoods.area_id = T.area_id;"""
pgexec (conn, sqlcmd, row, "Update neighbourhoods data")

# Step 4. Drop the tmp table
# DROP TABLE tmp_x;        
# pgexec (conn, insert_stmt, row, "row inserted")

# Adding index to tables

In [ ]:
sqlcmd = """CREATE INDEX area_name_result_ind ON walkability.result (area_name);"""
pgexec (conn, sqlcmd, row, "Create an index")
sqlcmd = """CREATE INDEX walkability_result_ind ON walkability.result (walkability);"""
pgexec (conn, sqlcmd, row, "Create an index")
sqlcmd = """CREATE INDEX gps_neighbourhoods_ind ON walkability.neighbourhoods (gps);"""
pgexec (conn, sqlcmd, row, "Create an index")
sqlcmd = "CREATE INDEX area_name_walkability_neighbourhoods_ind ON walkability.result(walkability,area_name);"
pgexec (conn, sqlcmd, row, "Create an index")